In [27]:
category_alias = {}
def categorize(category):
    if category in category_alias:
        return category_alias[category]
    if category_alias:
        category_alias[category] = max(category_alias.values()) + 1
    else:
        category_alias[category] = 0
    return category_alias[category]

def training(input_file, model='CBOW'):
    # Abrir el archivo de entrada y el archivo de salida
    with open(input_file, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        
    # Separar temas de palabras clave
    #corpus = []
    #for words in lines[1:]:
    #    synopsis, genre = words.strip().split('\t')
    #    corpus[genre] += (genre, synopsis.split(" "))
    
    ## Load a model to train
    if model == 'CBOW':
        from gensim.models import Word2Vec
        #model = Word2Vec(sentences=list(corpus.values()), sg = 0)
        #model.save("../data/word2vec-cbow.model")
        
    elif model == 'SG':
        from gensim.models import Word2Vec
        #model = Word2Vec(sentences=list(corpus.values()), sg = 1)
        #model.save("../data/word2vec-sg.model")
    elif model == "BAYES":
        import pandas as pd 
        df = pd.read_csv(input_file, delimiter='\t')
       
        ## Convertimos cada genero a un valor numérico
        df['genero'] = df['genero'].apply( lambda c : categorize(c) )
        
        #print(f"{df}")
        
        ## Entrenamiento, creando vectores por las palabras
        from sklearn.model_selection import train_test_split
        films_train, films_test, response_train, response_test = train_test_split( df.sinopsis, df.genero, test_size = .25)
        
        #print(f"{films_train}, {response_train}")
        
        from sklearn.feature_extraction.text import CountVectorizer
        vector = CountVectorizer()
        films_train_count = vector.fit_transform(films_train)
        
        #print(f"{films_train_count}")
        
        from sklearn.naive_bayes import MultinomialNB
        model = MultinomialNB()
        model.fit(films_train_count, response_train)
        
        return model
        
    else:
        print(f"ERROR unknown model {model}")

In [28]:
input_file = '../data/stemming_data.txt'
training(input_file, "BAYES")

                                              sinopsis  genero
0    eddi convenc tres amig jug ahorr part cart har...       0
1    stan corrupt agent dea cuy codici llev comet a...       1
2    cècil sonj richt joachim nikolaj lie kaas punt...       2
3    sobrellev insomni cronic sufr despues regres v...       1
4    manni ladron poc mont acept particip gran golp...       3
..                                                 ...     ...
445  tranquil local sureñ mississippi dos joven bor...       1
446  vid transexual oper felicity huffm experiment ...       2
447  grac amig judith sal dar pase caball sufr terr...       4
448  japon sigl xii kiot baj puert derru templ rash...       1
449  biograf jean-michel basquiat pintor vanguard p...       2

[450 rows x 2 columns]
11     emigr cuban fri sanguinari tony montan lleg cu...
12     curs italian une seis danes problem sentimenta...
389    will jov rebeld inteligent asombr especial mat...
377    virus mortal sid liber poblacion raccoon c

MultinomialNB()